In [1]:
from parsing import load_problem, load_solution
from graph_representation import Graph
import time

In [2]:
# weryfikuje poprawność sortowania
def verify_toposort(T, G):
    for i in xrange(len(T)):
        v = T[i]
        for s in G.graph[v].successors:
            s = (v[0] + s[0],) + s[1:]
            if s in T[:i]:
                return False
    return True

In [3]:
def longest_path(T, G, v1, v2, justH1=False):
    if v1 not in G.graph or v2 not in G.graph:
        return None
    temp_longest = {v1 : G.graph[v1].weight}
    
    print G.graph[v1].weight
    
    if v1 == v2:
        return temp_longest[v1]
    
    for v in T[T.index(v1)+1:]:
        if (not justH1) or v[0] == 0:
            print v
            preds = [p for p in G.graph[v].predecessors if p in temp_longest]
            print preds
            if preds:
                best_pred = max(preds, key=lambda p: temp_longest[p] + G.graph[p].weight)
                print best_pred
                temp_longest[v] = temp_longest[best_pred] + G.graph[best_pred].weight
            if v == v2:
                if v2 in temp_longest:
                    return temp_longest[v2]
                return None

In [4]:
# m_from, m_to - skąd zabieramy, gdzie wsadzamy
# op - zabierana operacja, para (nr_maszyny, nr_operacji)
# pos - pozycja do wsadzenia op na maszynę m_to

# tutaj numery maszyn idą od 0
def lower_bound(G, m_from, op, m_to, pos):
    def alpha(k,i):
        assert i >= 0
        if k != m_from or solution[k].index(op) > i:
            return G.solution[k][i]
        return G.solution[k][i+1]
    
    # jeśli longest_path nie istnieje, to powinien zwrócić None
    # te drogi poniżej są zawarte w H1 (pierwszej składowej G), więc
    # można by je liczyć trochę szybciej (tylko w H1)
    
    def R(k,opr):
        return G.longest_path((0,) + alpha(k,0), (0,) + opr, justH1=True)
    
    def Q(k,opr):
        return G.longest_path((0,) + opr, (0,) + alpha(k, len(G.solution[k])-1), justH1=True)

    def LB(l):
        op_old_weight = G.graph[(0,) + op].weight
        op_new_weight = G.problem[op[0]-1][op[1]-1][m_to]
        
        R1 = lambda: R(l, alpha(m_to, pos-1))
        R2 = lambda: R(l, op) - op_old_weight
        Q1 = lambda: Q(l, alpha(m_to, pos))
        Q2 = lambda: Q(l, op) - op_old_weight
        
        # 0 w maxach po to, żeby mieć jakąś liczbę, gdy drogi nie istnieją
        if pos == 0:
            return max(R2(), 0) + max(Q1(), Q2(), 0) + op_old_weight
        if pos == len(G.solution[k]):
            return max(R1(), R2(), 0) + max(Q2(), 0) + op_old_weight
        
        maxR = max(R1(), R2(), 0)
        maxQ = max(Q1(), Q2(), 0)
        
        return maxR + maxQ + op_old_weight # a może tu ma być op_new_weight?
    
    return max([LB(l) for l in xrange(len(solution))])

# JUŻ NIEAKTUALNE:
# co jeśli m_from == m_to (czy w ogóle może tak być?) - alpha(m_to, pos) może nie istnieć

In [5]:
n, m, problem = load_problem("instances/Barnes_mt10c1.fjs")
solution = load_solution("solutions/Barnes_mt10c1.txt")

G = Graph(n, m, problem, solution)
G.graph[(3,1,4)].successors
G.topological_sort()

In [6]:
G.graph[(0,9,1)].successors

[(0, 2, 1), (0, 9, 2)]

In [13]:
G.graph[(0,2,1)].successors

[(0, 7, 2), (0, 2, 2)]

In [14]:
G.graph[(0,9,2)].predecessors

[(0, 10, 1), (0, 9, 1)]

In [16]:
G.graph[(0,10,1)].predecessors

[(0, 7, 1)]

In [7]:
len(G.topological_order)

1200

In [20]:
G.longest_path_length((0,9,1), (0,2,1))

195

In [9]:
G.problem[8]

[{1: 76, 11: 76},
 {2: 69},
 {4: 76},
 {6: 51},
 {3: 85},
 {10: 11},
 {7: 40},
 {8: 89},
 {5: 26},
 {9: 74}]